In [31]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import argparse
import os.path
import sys
import time

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

# Basic model parameters as external flags.
FLAGS = None

parser = argparse.ArgumentParser()
parser.add_argument(
  '--learning_rate',
  type=float,
  default=0.01,
  help='Initial learning rate.'
)
parser.add_argument(
  '--max_steps',
  type=int,
  default=20000,
  help='Number of steps to run trainer.'
)
parser.add_argument(
  '--hidden1',
  type=int,
  default=128,
  help='Number of units in hidden layer 1.'
)
parser.add_argument(
  '--hidden2',
  type=int,
  default=32,
  help='Number of units in hidden layer 2.'
)
parser.add_argument(
  '--batch_size',
  type=int,
  default=100,
  help='Batch size.  Must divide evenly into the dataset sizes.'
)
parser.add_argument(
  '--input_data_dir',
  type=str,
  default='/tmp/tensorflow/mnist/input_data',
  help='Directory to put the input data.'
)
parser.add_argument(
  '--log_dir',
  type=str,
  default='/tmp/tensorflow/mnist/logs/fully_connected_feed',
  help='Directory to put the log data.'
)
parser.add_argument(
  '--fake_data',
  default=False,
  help='If true, uses fake data for unit testing.',
  action='store_true'
)

FLAGS, unparsed = parser.parse_known_args()

In [3]:
# Get the sets of images and labels for training, validation, and
# test on MNIST.
data_sets = input_data.read_data_sets("/tmp/tensorflow/mnist/input_data", False)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


In [9]:
images_placeholder = tf.placeholder(tf.float32, shape=(100,mnist.IMAGE_PIXELS))
labels_placeholder = tf.placeholder(tf.int32, shape=(100))

In [10]:
NUM_CLASSES = 10

# The MNIST images are always 28x28 pixels.
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE
def inference(images, hidden1_units, hidden2_units):
  """Build the MNIST model up to where it may be used for inference.
  Args:
    images: Images placeholder, from inputs().
    hidden1_units: Size of the first hidden layer.
    hidden2_units: Size of the second hidden layer.
  Returns:
    softmax_linear: Output tensor with the computed logits.
  """
  # Hidden 1
  with tf.name_scope('hidden1'):
    weights = tf.Variable(
        tf.truncated_normal([IMAGE_PIXELS, hidden1_units],stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
        name='weights')
    biases = tf.Variable(tf.zeros([hidden1_units]),name='biases')
    hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
  # Hidden 2
  with tf.name_scope('hidden2'):
    weights = tf.Variable(
        tf.truncated_normal([hidden1_units, hidden2_units],stddev=1.0 / math.sqrt(float(hidden1_units))),
        name='weights')
    biases = tf.Variable(tf.zeros([hidden2_units]),name='biases')
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
  # Linear
  with tf.name_scope('softmax_linear'):
    weights = tf.Variable(
        tf.truncated_normal([hidden2_units, NUM_CLASSES],stddev=1.0 / math.sqrt(float(hidden2_units))),
        name='weights')
    biases = tf.Variable(tf.zeros([NUM_CLASSES]),name='biases')
    logits = tf.matmul(hidden2, weights) + biases
  return logits

In [11]:
# Build a Graph that computes predictions from the inference model.
logits = mnist.inference(images_placeholder,
                         FLAGS.hidden1,
                         FLAGS.hidden2)

In [15]:
def loss(logits, labels):
  """Calculates the loss from the logits and the labels.
  Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size].
  Returns:
    loss: Loss tensor of type float.
  """
  labels = tf.to_int64(labels)
  cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels, name='xentropy')
  loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
  return loss

# Add to the Graph the Ops for loss calculation.
loss = mnist.loss(logits, labels_placeholder)

In [16]:
def training(loss, learning_rate):
  """Sets up the training Ops.
  Creates a summarizer to track the loss over time in TensorBoard.
  Creates an optimizer and applies the gradients to all trainable variables.
  The Op returned by this function is what must be passed to the
  `sess.run()` call to cause the model to train.
  Args:
    loss: Loss tensor, from loss().
    learning_rate: The learning rate to use for gradient descent.
  Returns:
    train_op: The Op for training.
  """
  # Add a scalar summary for the snapshot loss.
  tf.summary.scalar('loss', loss)
  # Create the gradient descent optimizer with the given learning rate.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  # Create a variable to track the global step.
  global_step = tf.Variable(0, name='global_step', trainable=False)
  # Use the optimizer to apply the gradients that minimize the loss
  # (and also increment the global step counter) as a single training step.
  train_op = optimizer.minimize(loss, global_step=global_step)
  return train_op

# Add to the Graph the Ops that calculate and apply gradients.
train_op = mnist.training(loss, FLAGS.learning_rate)

In [18]:
def evaluation(logits, labels):
  """Evaluate the quality of the logits at predicting the label.
  Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
  Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
  """
  # For a classifier model, we can use the in_top_k Op.
  # It returns a bool tensor with shape [batch_size] that is true for
  # the examples where the label is in the top k (here k=1)
  # of all logits for that example.
  correct = tf.nn.in_top_k(logits, labels, 1)
  # Return the number of true entries.
  return tf.reduce_sum(tf.cast(correct, tf.int32))

# Add the Op to compare the logits to the labels during evaluation.
eval_correct = mnist.evaluation(logits, labels_placeholder)

In [19]:
# Build the summary Tensor based on the TF collection of Summaries.
summary = tf.summary.merge_all()

In [20]:
# Add the variable initializer Op.
init = tf.global_variables_initializer()

In [21]:
# Create a saver for writing training checkpoints.
saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)

In [22]:
# Create a session for running Ops on the Graph.
sess = tf.Session()

In [23]:
# Instantiate a SummaryWriter to output summaries and the Graph.
summary_writer = tf.train.SummaryWriter(FLAGS.log_dir, sess.graph)

In [24]:
# Run the Op to initialize the variables.
sess.run(init)

In [26]:
def fill_feed_dict(data_set, images_pl, labels_pl):
  """Fills the feed_dict for training the given step.
  A feed_dict takes the form of:
  feed_dict = {
      <placeholder>: <tensor of values to be passed for placeholder>,
      ....
  }
  Args:
    data_set: The set of images and labels, from input_data.read_data_sets()
    images_pl: The images placeholder, from placeholder_inputs().
    labels_pl: The labels placeholder, from placeholder_inputs().
  Returns:
    feed_dict: The feed dictionary mapping from placeholders to values.
  """
  # Create the feed_dict for the placeholders filled with the next
  # `batch size` examples.
  images_feed, labels_feed = data_set.next_batch(FLAGS.batch_size,
                                                 FLAGS.fake_data)
  feed_dict = {
      images_pl: images_feed,
      labels_pl: labels_feed,
  }
  return feed_dict

def do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_set):
  """Runs one evaluation against the full epoch of data.
  Args:
    sess: The session in which the model has been trained.
    eval_correct: The Tensor that returns the number of correct predictions.
    images_placeholder: The images placeholder.
    labels_placeholder: The labels placeholder.
    data_set: The set of images and labels to evaluate, from
      input_data.read_data_sets().
  """
  # And run one epoch of eval.
  true_count = 0  # Counts the number of correct predictions.
  steps_per_epoch = data_set.num_examples // FLAGS.batch_size
  num_examples = steps_per_epoch * FLAGS.batch_size
  for step in xrange(steps_per_epoch):
    feed_dict = fill_feed_dict(data_set,
                               images_placeholder,
                               labels_placeholder)
    true_count += sess.run(eval_correct, feed_dict=feed_dict)
  precision = true_count / num_examples
  print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
        (num_examples, true_count, precision))

In [32]:
# Start the training loop.
for step in xrange(FLAGS.max_steps):
  start_time = time.time()

  # Fill a feed dictionary with the actual set of images and labels
  # for this particular training step.
  feed_dict = fill_feed_dict(data_sets.train,
                             images_placeholder,
                             labels_placeholder)

  # Run one step of the model.  The return values are the activations
  # from the `train_op` (which is discarded) and the `loss` Op.  To
  # inspect the values of your Ops or variables, you may include them
  # in the list passed to sess.run() and the value tensors will be
  # returned in the tuple from the call.
  _, loss_value = sess.run([train_op, loss],
                           feed_dict=feed_dict)

  duration = time.time() - start_time

  # Write the summaries and print an overview fairly often.
  if step % 100 == 0:
    # Print status to stdout.
    print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
    # Update the events file.
    summary_str = sess.run(summary, feed_dict=feed_dict)
    summary_writer.add_summary(summary_str, step)
    summary_writer.flush()

  # Save a checkpoint and evaluate the model periodically.
  if (step + 1) % 1000 == 0 or (step + 1) == FLAGS.max_steps:
    checkpoint_file = os.path.join(FLAGS.log_dir, 'model.ckpt')
    saver.save(sess, checkpoint_file, global_step=step)
    # Evaluate against the training set.
    print('Training Data Eval:')
    do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_sets.train)
    # Evaluate against the validation set.
    print('Validation Data Eval:')
    do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_sets.validation)
    # Evaluate against the test set.
    print('Test Data Eval:')
    do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_sets.test)

Step 0: loss = 0.30 (0.021 sec)
Step 100: loss = 0.13 (0.002 sec)


Step 200: loss = 0.20 (0.002 sec)


Step 300: loss = 0.13 (0.120 sec)
Step 400: loss = 0.13 (0.002 sec)


Step 500: loss = 0.20 (0.002 sec)
Step 600: loss = 0.13 (0.002 sec)


Step 700: loss = 0.15 (0.002 sec)
Step 800: loss = 0.10 (0.002 sec)


...

Step 19400: loss = 0.04 (0.002 sec)
Step 19500: loss = 0.07 (0.002 sec)


Step 19600: loss = 0.11 (0.002 sec)
Step 19700: loss = 0.06 (0.002 sec)


Step 19800: loss = 0.05 (0.002 sec)
Step 19900: loss = 0.03 (0.002 sec)


Training Data Eval:


  Num examples: 55000  Num correct: 54124  Precision @ 1: 0.9841
Validation Data Eval:
  Num examples: 5000  Num correct: 4854  Precision @ 1: 0.9708
Test Data Eval:
  Num examples: 10000  Num correct: 9727  Precision @ 1: 0.9727
